In [ ]:
#Import required modules

from arcgis.gis import GIS
import pandas as pd
import datetime
import time
import pytz

In [ ]:
#Connect to your GIS
#Update the url, username, and password
gis = GIS(url='portalurl', username='admin', password='admin pw')

#Connect to your ArcGIS Server
ags = gis.admin.servers.list()[0]

#Verify connection
ags

In [ ]:
#Function for grabbing the server logs by server folder

def get_logs(folder):
    # Set the current time and calculate the start time and end time
    now = datetime.datetime.now()
    starttime = now - datetime.timedelta(days=0)
    endtime = now - datetime.timedelta(days=2)

    # Define timezones
    us_central_timezone = pytz.timezone('US/Central')

    # Measure the time taken to query the logs
    query_start_time = time.time()
    print('Querying Logs')
    recent_logs = ags.logs.query(level='WARNING',start_time=starttime, end_time=endtime)
    query_end_time = time.time()
    query_time = query_end_time - query_start_time

    # Measure the time taken to process the logs
    processing_start_time = time.time()
    
    #Process logs
    logs = []
    for log in recent_logs['logMessages']:
        if log['source'].split('/')[0] == folder:
            value = log['time']
            utc_datetime = datetime.datetime.utcfromtimestamp(value / 1000.0).replace(tzinfo=pytz.UTC)
            us_central_datetime = utc_datetime.astimezone(us_central_timezone)
            date = us_central_datetime.strftime('%m/%d/%Y %H:%M:%S')
            logs.append((log['type'], date, log['source'], log['message'], log['user'], log['machine'], log['code'], log['methodName'], log['requestID']))

    processing_end_time = time.time()
    processing_time = processing_end_time - processing_start_time

    # Print the times
    print(f"Time taken to query the logs: {query_time:.2f} seconds")
    print(f"Time taken to process the logs: {processing_time:.2f} seconds")

    # Create the Dataframe
    columns = ['Type', 'Date', 'Source', 'Message', 'User', 'Machine', 'Code', 'Method Name', 'Request ID']
    df = pd.DataFrame(logs, columns = columns)
    server_url = ags.url.split('//')[1]
    web_url = server_url.split('/')[0]
    
    #Update this variable to save to a difference loc
    folder_dir = r"C:\"
    
    #Spreadsheet name
    xcel = f"{folder_dir}/{web_url}-{folder}.xlsx"
    print(xcel)
    
    #export to excel
    df.to_excel(xcel, index = False)
    
    return df

In [ ]:
#Execture the function. Update the folder with a folder from your ArcGIS Server
df = get_logs('Folder Name Here')
print(f"There are {len(df)} log messages.")

In [ ]:
#Display the dataframe
df.head(-5)